In [ ]:
col_new = ['ratio_click_of_aid_in_uid',
 'ratio_click_of_creativeSize_in_uid',
 'ratio_click_of_age_in_aid',
 'ratio_click_of_age_in_creativeSize',
 'ratio_click_of_gender_in_advertiserId',
 'ratio_click_of_gender_in_creativeSize',
 'ratio_click_of_consumptionAbility_in_aid',
 'ratio_click_of_age_in_advertiserId',
 'ratio_click_of_productType_in_uid',
 'ratio_click_of_productType_in_consumptionAbility',
 'ratio_click_of_productType_in_age',
 'ratio_click_of_gender_in_consumptionAbility',
 'ratio_click_of_creativeSize_in_age',
 'ratio_click_of_gender_in_aid',
 'ratio_click_of_creativeSize_in_productType',
 'ratio_click_of_house_in_campaignId',
 'ratio_click_of_house_in_creativeSize',
 'ratio_click_of_aid_in_creativeSize',
 'ratio_click_of_productId_in_uid',
 'ratio_click_of_os_in_advertiserId',
 'ratio_click_of_adCategoryId_in_uid',
 'ratio_click_of_productType_in_creativeSize',
 'ratio_click_of_productType_in_os',
 'ratio_click_of_productType_in_education',
 'ratio_click_of_advertiserId_in_uid',
 'ratio_click_of_gender_in_productId',
 'ratio_click_of_consumptionAbility_in_age',
 'ratio_click_of_adCategoryId_in_creativeSize',
 'ratio_click_of_creativeSize_in_education',
 'ratio_click_of_campaignId_in_uid',
 'ratio_click_of_consumptionAbility_in_advertiserId']
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import os
import numpy as np
import time
import random
import warnings
warnings.filterwarnings("ignore")
##读取数据
print("Reading...")
data = pd.read_csv('train_test_merge.csv')
print('Index...')
train_part_index = list(data[(data['label']!=-1)&(data['n_parts']!=1)].index)
evals_index = list(data[(data['label']!=-1)&(data['n_parts']==1)].index)
test1_index = list(data[data['n_parts']==6].index)
test2_index = list(data[data['n_parts']==7].index)
print('LabelEncoder...')
label_feature=['aid','uid', 'advertiserId', 'campaignId', 'creativeId',
       'creativeSize', 'adCategoryId', 'productId', 'productType', 'age',
       'gender','education', 'consumptionAbility', 'LBS',
       'os', 'carrier', 'house']
data = data[label_feature]
for feature in label_feature:
    s = time.time()
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])
    print(feature,int(time.time()-s),'s')
print('Done')


df_feature = pd.DataFrame()
data['cnt']=1
num = 0
print('Ratio clcik...')
n = len(col_type)
for i in range(n):
    col_name = "ratio_click_of_"+col_type[i]
    if col_name in col_new:
        s = time.time()
        df_feature[col_name] =(data[col_type[i]].map(data[col_type[i]].value_counts())/len(data)*100).astype(int)
        num+=1
        print(num,col_name,int(time.time()-s),'s')
n = len(col_type)
for i in range(n):
    for j in range(n):
        if i!=j:
            col_name = "ratio_click_of_"+col_type[j]+"_in_"+col_type[i]
            if col_name in col_new:
                s = time.time()
                se = data.groupby([col_type[i],col_type[j]])['cnt'].sum()
                dt = data[[col_type[i],col_type[j]]]
                cnt = data[col_type[i]].map(data[col_type[i]].value_counts())
                df_feature[col_name] = ((pd.merge(dt,se.reset_index(),how='left',on=[col_type[i],col_type[j]]).sort_index()['cnt'].fillna(value=0)/cnt)*100).astype(int).values
                num+=1
                print(num,col_name,int(time.time()-s),'s')
print('Saving...')
print('train_part...')
df_feature.loc[train_part_index].to_csv('train_part_x_ratio.csv',index=False)
print('evals...')
df_feature.loc[evals_index].to_csv('evals_x_ratio.csv',index=False)
print('test1...')
df_feature.loc[test1_index].to_csv('test1_x_ratio.csv',index=False)
print('test2...')
df_feature.loc[test2_index].to_csv('test2_x_ratio.csv',index=False)
print('Over')

In [13]:
17*16/2+17

153.0